# StreamBasedCache Demo - New York Taxi Rides

## Install dependencies

In [ ]:
try:
    import google.colab
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

## Imports

In [ ]:
from __future__ import division, print_function

import atexit
import contextlib
import gc
import itertools
import json
import os
import time
import uuid
from datetime import datetime

import apache_beam as beam
import dateutil
import pytz
from apache_beam import transforms
from apache_beam.io.gcp.pubsub import PubsubMessage
from apache_beam.options.pipeline_options import (GoogleCloudOptions,
                                                  PipelineOptions)
from apache_beam.runners.interactive.caching import streambasedcache
from apache_beam.transforms import combiners, window
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from google.api_core import exceptions as gexc
from google.cloud import pubsub_v1

try:
    from contextlib import ExitStack
except ImportError:
    from contextlib2 import ExitStack

## Parameters

In [ ]:
NOTEBOOK_NAME = "streambasedcache-new_york_taxirides"

try:
    os.makedirs(NOTEBOOK_NAME)
except OSError:
    pass

In [ ]:
# { display-mode: "form" }
project_id = "strokach-playground"  #@param {type:"string"}
remote_temp_location = "gs://strokach/dataflow_temp"  #@param {type:"string"}
taxirides_topic_path = "projects/pubsub-public-data/topics/taxirides-realtime"  #@param {type:"string"}

In [ ]:
options = PipelineOptions(
    project=project_id,
    temp_location=remote_temp_location,
    streaming=True,
    runner="DirectRunner",
#     runner="DataflowRunner",
    sdk_location=os.path.expanduser(
        "~/workspace/beam/sdks/python/dist/apache-beam-2.15.0.dev0.tar.gz"
    ),
    setup_file="../setup.py",
    job_name="test7",
)
options.display_data()

In [ ]:
if GOOGLE_COLAB:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception:
    print("No autoreload")

## Function definitions

In [ ]:
def create_cache(location, cache_class, *args, **kwargs):
    for _ in range(3):
        full_location = "{}-{}".format(location, uuid.uuid4().hex[:8])
        try:
            return cache_class(full_location, *args, **kwargs)
        except IOError as e:
            pass
    raise e

In [ ]:
@contextlib.contextmanager
def run_pipeline(p):
    pr = p.run()
    try:
        yield pr
    finally:
        pr.cancel()

### Create subscription

In [ ]:
sub_client = pubsub_v1.SubscriberClient()

In [ ]:
@contextlib.contextmanager
def create_pubsub_subscription(project_id, topic_path):
    short_sub_name = topic_path.split("/")[-1] + "-" + uuid.uuid4().hex
    subscription_path = sub_client.subscription_path(project_id, short_sub_name)
    sub_client.create_subscription(subscription_path, topic_path)
    try:
        yield subscription_path
    finally:
        sub_client.delete_subscription(subscription_path)

In [ ]:
class LoadPubSubMessage(beam.DoFn):
    def process(self, message):
        from datetime import datetime
        import json

        import arrow
        import pytz

        from apache_beam.transforms import window
        from apache_beam.utils import timestamp

        timestamp_str = message.attributes["ts"]
        dt = arrow.get(timestamp_str).datetime
        dt_delta = dt.astimezone(pytz.UTC) - datetime.utcfromtimestamp(0).replace(
            tzinfo=pytz.UTC
        )
        timestamp = timestamp.Timestamp(seconds=dt_delta.total_seconds())

        element = json.loads(message.data.decode())
        element["timestamp_milliseconds"] = int(timestamp.micros / 1000)
        yield window.TimestampedValue(element, timestamp)


message = PubsubMessage(
    data=json.dumps({"x": "hello"}).encode("utf-8"),
    attributes={"ts": "2019-06-27T20:36:35.4972-04:00"},
)
assert next(LoadPubSubMessage().process(message)).value == {
    u"x": u"hello",
    "timestamp_milliseconds": 1561682195497,
}
assert next(LoadPubSubMessage().process(message)).timestamp.micros == 1561682195497200
next(LoadPubSubMessage().process(message)).value

In [ ]:
class AddMercatorCoords(beam.DoFn):
    
    def process(self, element):
        import numpy as np

        r_major = 6378137.000

        element["utm_x"] = (r_major * 2 * np.pi / 360) * element["longitude"]
        try:
            scale = element["utm_x"] / element["longitude"]
        except ZeroDivisionError:
            scale = 0
        element["utm_y"] = (
            180.0 / np.pi * np.log(np.tan((np.pi / 4.0) + element["latitude"] * (np.pi / 180.0 / 2.0))) * scale
        )
        yield element
#         events_df["utm_y"] = events_df["utm_y"].fillna(0)
        
        
next(AddMercatorCoords().process({"longitude": 0, "latitude": 0}))

In [ ]:
class FilterByRegion(beam.DoFn):
    def __init__(self, utm_x_range, utm_y_range):
        self.utm_x_range = utm_x_range
        self.utm_y_range = utm_y_range

    def process(self, element):
        if (self.utm_x_range[0] <= element["utm_x"] < self.utm_x_range[1]) and (
            self.utm_y_range[0] <= element["utm_y"] < self.utm_y_range[1]
        ):
            yield element

In [ ]:
class FilterRideStatus(beam.DoFn):
    def __init__(self, ride_status):
        self._ride_status = ride_status
        super(FilterRideStatus, self).__init__()

    def process(self, element):
        if element["ride_status"] in self._ride_status:
            yield element

In [ ]:
class AddWindowRange(beam.DoFn):
    def process(
        self, element, window=beam.DoFn.WindowParam, timestamp=beam.DoFn.TimestampParam
    ):
        import pytz

        element = {"events": element}

        ts_format = "%Y-%m-%dT%H:%M:%S.%f-04:00"
        element["window_start_est"] = (
            window.start.to_utc_datetime()
            .replace(tzinfo=pytz.UTC)
            .astimezone(pytz.timezone("US/Eastern"))
            .strftime(ts_format)
        )
        element["window_end_est"] = (
            window.end.to_utc_datetime()
            .replace(tzinfo=pytz.UTC)
            .astimezone(pytz.timezone("US/Eastern"))
            .strftime(ts_format)
        )
        element["window_start_micros"] = window.start.micros
        element["window_end_micros"] = window.end.micros
        element["timestamp_micros"] = timestamp.micros
        yield element

In [ ]:
class DumpPubsubMessage(beam.DoFn):
    def __init__(self):
        pass

    def process(
        self, element, window=beam.DoFn.WindowParam, timestamp=beam.DoFn.TimestampParam
    ):
        from apache_beam.io.gcp.pubsub import PubsubMessage

        data = element
        attributes = {"ts": str(int(timestamp.micros / 1000.0))}

        message = PubsubMessage(json.dumps(data).encode("utf-8"), attributes)
        yield message

### Plotting

In [ ]:
import bokeh
from bokeh.core.properties import value
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, Label, LabelSet, Legend, LegendItem, Range1d
from bokeh.models.annotations import Title
from bokeh.plotting import figure, show
from bokeh.tile_providers import Vendors, get_provider
from bokeh.transform import factor_cmap


def create_map():
    # Colormap
    # cmap = bokeh.palettes.d3["Category10"][4]
    cmap = bokeh.palettes.d3["Category20b"][20]
    colors = [cmap[1], cmap[9], cmap[-2]]

    # Source of data
    source = ColumnDataSource(data=dict(x=[], y=[], ride_status=[]))

    # Background map
    fg = figure(
        x_range=MERCATOR_X_RANGE,
        y_range=MERCATOR_Y_RANGE,
        x_axis_type="mercator",
        y_axis_type="mercator",
        title_location="above",
        plot_height=600,
    )
    fg.add_tile(get_provider(Vendors.CARTODBPOSITRON))

    # Scatterplot
    fg.circle(
        x="x",
        y="y",
        source=source,
        size=2,
        color=factor_cmap("ride_status", colors, ["pickup", "enroute", "dropoff"]),
        fill_alpha=0.8,
        #     legend=value("start", "stop"),
        #     legend=[value(x) for x in ["start", "stop"]],
    )

    # Legend
    pickup = fg.circle(x=[], y=[], color=colors[0])
    enroute = fg.circle(x=[], y=[], color=colors[1])
    dropoff = fg.circle(x=[], y=[], color=colors[2])
    legend = Legend(
        items=[("pickup", [pickup]), ("enroute", [enroute]), ("dropoff", [dropoff])]
    )
    fg.add_layout(legend)
    return fg, source

## Pipeline

In [ ]:
MERCATOR_X_RANGE = (-8240000, -8220000)
MERCATOR_Y_RANGE = (4950000, 5000000)

In [ ]:
try:
    stack.__exit__(None, None, None)
except NameError:
    pass

In [ ]:
try:
    stack.__exit__(None, None, None)
except NameError:
    pass

stack = ExitStack()
subscription_path = stack.enter_context(
    create_pubsub_subscription(project_id, taxirides_topic_path)
)

temp = create_cache(
    "projects/{}/topics/{}".format(project_id, NOTEBOOK_NAME),
    streambasedcache.PubSubBasedCache,
    with_attributes=True,
    timestamp_attribute="ts",
)

p = beam.Pipeline(options=options)

out = (
    p
    | "Read"
    >> beam.io.ReadFromPubSub(
        subscription=subscription_path, with_attributes=True, timestamp_attribute="ts"
    )
    | beam.Map(lambda message: json.loads(message.data.decode()))
    | "Add Mercator coords" >> beam.ParDo(AddMercatorCoords())
    | "Filter to New York"
    >> beam.ParDo(FilterByRegion(MERCATOR_X_RANGE, MERCATOR_Y_RANGE))
    | "Subsample" >> beam.Filter(lambda e: e["ride_id"][0] == "a")
    | "Window" >> beam.WindowInto(window.FixedWindows(5))
    | "Combine" >> beam.CombineGlobally(combiners.ToListCombineFn()).without_defaults()
    | "Add window info" >> beam.ParDo(AddWindowRange())
    #     | beam.Map(lambda e: print(e) or e)
    | "To PubSub" >> beam.ParDo(DumpPubsubMessage())
    | "Write" >> temp.writer()
)

pr = stack.enter_context(run_pipeline(p))

In [ ]:
# out = list(itertools.islice(temp.read(from_start=False, timeout=10, return_timestamp=True), 1))
# out

In [ ]:
gc.collect()

In [ ]:
output_notebook()

fg, source = create_map()
fg.title.text = "-"
fg.title.align = "center"

# Updates
handle = show(fg, notebook_handle=True)

days_of_week = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

jx = True
for timestamp, message in temp.read(
    from_start=True, timeout=200, return_timestamp=True, burnin=15
):
    if jx:
        time.sleep(15)
        jx = False
        continue
    data = json.loads(message.data.decode("utf-8"))["events"]
    dt = (
        datetime.utcfromtimestamp(timestamp.micros / 1000.0 / 1000.0 + 0.001)
        .replace(tzinfo=pytz.UTC)
        .astimezone(pytz.timezone("US/Eastern"))
    )
    dt_str = days_of_week[dt.weekday()] + " " + dt.strftime("%b %d %Y %I:%M:%S %f")
    fg.title.text = dt_str
    fg.title.align = "center"
    source.data = {
        "x": [d["utm_x"] for d in data],
        "y": [d["utm_y"] for d in data],
        "ride_status": [d["ride_status"] for d in data],
    }
    push_notebook(handle=handle)

    time.sleep(0.1)

In [ ]:
output_notebook()

fg, source = create_map()
fg.title.text = "-"
fg.title.align = "center"

# Updates
handle = show(fg, notebook_handle=True)

days_of_week = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

prev_timestamp = 0
data = []
for timestamp, message in temp.read(
    from_start=False, timeout=10, return_timestamp=True, burnin=3
):
    if timestamp.micros == prev_timestamp:
        chunk = json.loads(message.data.decode("utf-8"))["events"]
        data += chunk
        continue
    if timestamp.micros < prev_timestamp:
        print("Skipping {}".format(timestamp.micros))
        continue
    assert timestamp.micros > prev_timestamp
    dt = (
        datetime.utcfromtimestamp(prev_timestamp / 1000.0 / 1000.0 + 0.001)
        .replace(tzinfo=pytz.UTC)
        .astimezone(pytz.timezone("US/Eastern"))
    )
    dt_str = days_of_week[dt.weekday()] + " " + dt.strftime("%b %d %Y %I:%M:%S %f")
    fg.title.text = dt_str
    fg.title.align = "center"
    source.data = {
        "x": [d["utm_x"] for d in data],
        "y": [d["utm_y"] for d in data],
        "ride_status": [d["ride_status"] for d in data],
    }
    push_notebook(handle=handle)
    
    data = json.loads(message.data.decode("utf-8"))["events"]
    prev_timestamp = timestamp.micros

    time.sleep(0.3)

In [ ]:
try:
    directrunner_stack.__exit__(None, None, None)
except NameError:
    pass

directrunner_stack = ExitStack()
subscription_path = directrunner_stack.enter_context(
    create_pubsub_subscription(project_id, taxirides_topic_path)
)

directrunner_temp = create_cache(
    "projects/{}/topics/{}".format(project_id, NOTEBOOK_NAME),
    streambasedcache.PubSubBasedCache,
    with_attributes=True,
    timestamp_attribute="ts",
)

p = beam.Pipeline(options=options)

out = (
    p
    | "Read"
    >> beam.io.ReadFromPubSub(
        subscription=subscription_path, with_attributes=True, timestamp_attribute="ts"
    )
    | beam.Map(lambda message: json.loads(message.data.decode()))
    | "Add Mercator coords" >> beam.ParDo(AddMercatorCoords())
    | "Filter to New York"
    >> beam.ParDo(FilterByRegion(MERCATOR_X_RANGE, MERCATOR_Y_RANGE))
    | "Subsample" >> beam.Filter(lambda e: e["ride_id"][0] == "a")
    | "Window" >> beam.WindowInto(window.FixedWindows(5))
    | "Combine" >> beam.CombineGlobally(combiners.ToListCombineFn()).without_defaults()
    | "Add window info" >> beam.ParDo(AddWindowRange())
    #     | beam.Map(lambda e: print(e) or e)
    | "To PubSub" >> beam.ParDo(DumpPubsubMessage())
    | "Write" >> temp.writer()
)

pr = directrunner_stack.enter_context(run_pipeline(p))

In [ ]:
try:
    print(pr.cancel())
except NameError:
    pass

In [ ]:
random.randint(0, 10)

In [ ]:
import threading
output_notebook()

fg = figure(plot_width=400, plot_height=400)

source = ColumnDataSource(
    data=dict(x=[1, 2, 3, 4, 5], y=[random.randint(0, 10) for _ in range(5)])
)

# add a circle renderer with a size, color, and alpha
fg.circle("x", "y", size=20, color="navy", alpha=0.5, source=source)

# show the results
handle = show(fg, notebook_handle=True)

def update_plot():
    while True:
        time.sleep(1)
        source.data = {"x": [1, 2, 3, 4, 5], "y": [random.randint(0, 10) for _ in range(5)]}
        #     print(source.data)
        push_notebook(handle=handle)
        
        
t2 = threading.Thread(target=update_plot)
t2.start()

In [ ]:
t1.join()

In [ ]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_notebook()

fg = figure(plot_width=400, plot_height=400)

source = ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[6, 7, 2, 4, 5]))

# add a circle renderer with a size, color, and alpha
fg.circle("x", "y", size=20, color="navy", alpha=0.5, source=source)

# show the results
handle = show(fg, notebook_handle=True)

while True:
    time.sleep(1)
    source.data = {
        "x": source.data["x"] + [x + 1 for x in source.data["x"]],
        "y": [y + 1 for y in source.data["y"]],
    }
    #     print(source.data)
    push_notebook(handle=handle)

In [ ]:
try:
    data_source  = temp.read(timeout=10, return_timestamp=True)
#     time.sleep(3)
    out = list(itertools.islice(data_source, 100))
finally:
    pr.cancel()
out

In [ ]:
out = (
    p
    | "Read"
    >> beam.io.ReadFromPubSub(subscription=subscription_path, with_attributes=True)
    | "Decode message" >> beam.ParDo(LoadPubSubMessage())
    | "Window" >> beam.WindowInto(window.FixedWindows(1))
    | "Add window info" >> beam.ParDo(AddWindowRange())
    | "Combine" >> beam.CombineGlobally(combiners.ToListCombineFn()).without_defaults()
    | "Reduce"
    >> beam.Map(
        lambda e: {
            "timestamp_milliseconds": e[0]["window_end_milliseconds"],
            "number_of_events": len(e),
        }
    )
    | "Write" >> temp.writer()
)

In [ ]:
    #     | "Filter to pickups and dropoff"
    #     >> beam.ParDo(FilterRideStatus(["pickup", "dropoff"]))
    | "Pair with end of window" >> beam.Map(lambda e: (e["window_end_milliseconds"], e))
    | "Group by end of window" >> beam.GroupByKey()
    | "Reduce"
    >> beam.Map(
        lambda e: {"timestamp_milliseconds": e[0], "number_of_events": len(e[1])}
    )


In [ ]:
# [Timestamp(1562888333.275000),
#  Timestamp(1562888333.279000),
#  Timestamp(1562888333.286000),
#  Timestamp(1562888333.288000),
#  Timestamp(1562888333.294000)]

In [ ]:
# try:
#     pr.cancel()
# except NameError:
#     pass

### Interactive dashboard

In [ ]:
import bokeh
from bokeh.core.properties import value
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, Label, LabelSet, Legend, LegendItem, Range1d
from bokeh.models.annotations import Title
from bokeh.plotting import figure, show
from bokeh.tile_providers import Vendors, get_provider
from bokeh.transform import factor_cmap

if GOOGLE_COLAB:
    print("Interactive plot does not work on colab yet!")


def create_map():
    # Colormap
    # cmap = bokeh.palettes.d3["Category10"][4]
    cmap = bokeh.palettes.d3["Category20b"][20]
    colors = [cmap[1], cmap[9], cmap[-2]]

    # Source of data
    source = ColumnDataSource(data=dict(x=[], y=[], ride_status=[]))

    # Background map
    fg = figure(
        x_range=MERCATOR_X_RANGE,
        y_range=MERCATOR_Y_RANGE,
        x_axis_type="mercator",
        y_axis_type="mercator",
        title_location="above",
        plot_height=600,
    )
    fg.add_tile(get_provider(Vendors.CARTODBPOSITRON))

    # Scatterplot
    fg.circle(
        x="x",
        y="y",
        source=source,
        size=2,
        color=factor_cmap("ride_status", colors, ["pickup", "enroute", "dropoff"]),
        fill_alpha=0.8,
        #     legend=value("start", "stop"),
        #     legend=[value(x) for x in ["start", "stop"]],
    )

    # Legend
    pickup = fg.circle(x=[], y=[], color=colors[0])
    enroute = fg.circle(x=[], y=[], color=colors[1])
    dropoff = fg.circle(x=[], y=[], color=colors[2])
    legend = Legend(
        items=[("pickup", [pickup]), ("enroute", [enroute]), ("dropoff", [dropoff])]
    )
    fg.add_layout(legend)
    return fg, source

In [ ]:
output_notebook()

fg, source = create_map()

# Updates
handle = show(fg, notebook_handle=True)

days_of_week = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

data_source = temp.read(timeout=10, return_timestamp=True)
time.sleep(5)

for timestamp, message in data_source:
    data = message.data.data
    dt = datetime.utcfromtimestamp(timestamp).replace(
        tzinfo=pytz.UTC
    )  # .astimezone(pytz.timezone('US/Eastern'))
    dt_str = days_of_week[dt.weekday()] + " " + dt.strftime("%b %d %Y %I:%M:%S %f")
    fg.title.text = dt_str
    fg.title.align = "center"
    source.data = {
        "x": [d["utm_x"] for d in data],
        "y": [d["utm_y"] for d in data],
        "ride_status": [d["ride_status"] for d in data],
    }
#     print(source.data)
    push_notebook(handle=handle)
    time.sleep(0.1)